In [ ]:
# !pip install pypsa
# !pip install neptune-client
# !pip install gymnasium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.3/267.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 141.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.1 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.1.1
    Uninstalling shapely-2.1.1:
      Successfully uninstalled shapely-2.1.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 127.4 MB/s eta 0:00:00
   ━━━━━━━━━━

In [ ]:
# #Install debugger
# !pip install ipdb
# import ipdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.5 MB/s eta 0:00:00


In [1]:
import pypsa
import pandas as pd
import numpy as np
import gymnasium as gym
from gymnasium import spaces

import gc
import psutil
import matplotlib.pyplot as plt

import neptune

from torch.utils.data import TensorDataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F

/Users/antoniagrindrod/miniconda3/envs/pypsa-earth-rl/lib/python3.11/site-packages/swagger_spec_validator/validator12.py:18: DeprecationWarning: jsonschema.RefResolver is deprecated as of v4.18.0, in favor of the https://github.com/python-jsonschema/referencing library, which provides more compliant referencing behavior as well as more flexible APIs for customization. A future release will remove RefResolver. Please file a feature request (on referencing) if you are missing an API for the kind of customization you need.
  from jsonschema import RefResolver
/Users/antoniagrindrod/miniconda3/envs/pypsa-earth-rl/lib/python3.11/site-packages/swagger_spec_validator/validator12.py:18: DeprecationWarning: jsonschema.RefResolver is deprecated as of v4.18.0, in favor of the https://github.com/python-jsonschema/referencing library, which provides more compliant referencing behavior as well as more flexible APIs for customization. A future release will remove RefResolver. Please file a feature re

In [12]:
# Upload files to Google Drive first, then:
from google.colab import drive
drive.mount('/content/drive')

# Access your files
%cd /content/drive/MyDrive/Colab_Notebooks/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab_Notebooks


In [ ]:


    def debug_action_scaling(self, action):
        """Debug helper to check action scaling"""
        print("=== Action Scaling Debug ===")
        print(f"Input action shape: {action.shape}")
        print(f"Input action range: [{action.min():.3f}, {action.max():.3f}]")

        try:
            scaled = self.scale_action(action)
            print("Scaled actions:")
            for component, values in scaled.items():
                if len(values) > 0:
                    print(f"  {component}: shape={values.shape}, range=[{values.min():.3f}, {values.max():.3f}]")
                    print(f"    values: {values}")
                else:
                    print(f"  {component}: empty array")
        except Exception as e:
            print(f"Action scaling failed: {e}")
            import traceback
            traceback.print_exc()

    def validate_network_structure(self):
        """Validate that your network has the expected structure"""
        print("=== Network Structure Validation ===")

        # Check generators
        print(f"Total generators: {len(self.network.generators)}")
        print(f"Generator names: {list(self.network.generators.index)}")

        if hasattr(self.network.generators_t, 'p_max_pu'):
            print(f"Renewable generators (have p_max_pu): {list(self.network.generators_t.p_max_pu.columns)}")

        # Check storage units
        print(f"Total storage units: {len(self.network.storage_units)}")
        if len(self.network.storage_units) > 0:
            print(f"Storage unit names: {list(self.network.storage_units.index)}")
            print(f"Storage unit capacities: {self.network.storage_units.p_nom.values}")

        # Check loads
        print(f"Total loads: {len(self.network.loads)}")
        print(f"Load names: {list(self.network.loads.index)}")

        # Check snapshots
        print(f"Total snapshots: {len(self.network.snapshots)}")
        print(f"First few snapshots: {self.network.snapshots[:5].tolist()}")

In [11]:
# Simple test to run immediately
def quick_test():
    """Quick test you can run to check basic functionality"""
    try:
        # Update with your network file path
        network_file = '/content/drive/MyDrive/Colab_Notebooks/networks_1_week_connected/elec_s_10_ec_lc1.0_1h.nc'


        print("Creating environment...")
        env = Env2Gen1LoadConstr(network_file=network_file, episode_length=3)

        print("Validating network structure...")
        env.validate_network_structure()

        print("Testing reset...")
        obs, info = env.reset()
        print(f"Reset successful - observation shape: {obs.shape}")

        print("Testing single step...")
        action = env.action_space.sample()
        env.debug_action_scaling(action)

        obs, reward, terminated, truncated, info = env.step(action)
        print(f"Step successful - reward: {reward:.4f}")
        print(f"Constraints satisfied: {info.get('constraints_satisfied', 'N/A')}")

        print("✓ Quick test passed!")

    except Exception as e:
        print(f"✗ Quick test failed: {e}")
        import traceback
        traceback.print_exc()

In [25]:
quick_test()

Creating environment...


Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA0 5', 'ZA0 6', 'ZA0 7',
       'ZA0 8', 'ZA0 9', 'ZA1 0', 'ZA2 0', 'ZA3 0'],
      dtype='object', name='Bus')
Index(['0', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6',
       '7', '8', '9', 'lines new ZA0 4 <-> ZA2 0 AC',
       'lines new ZA0 0 <-> ZA1 0 AC', 'lines new ZA0 0 <-> ZA3 0 AC'],
      dtype='object', name='Line')
/usr/local/lib/python3.12/dist-packages/linopy/expressions.py:1861: FutureWarning:

In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): '_term' ('_term',) The recommendation is to set join explicitly for this case.



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded variable mapping from: /content/drive/MyDrive/Colab_Notebooks/var_constraint_map/elec_s_10_ec_lc1.0_1h_var_id_to_name.pkl
Loaded constraints from: /content/drive/MyDrive/Colab_Notebooks/var_constraint_map/elec_s_10_ec_lc1.0_1h_constraints.pkl
Validating network structure...
=== Network Structure Validation ===
Total generators: 55
Generator names: ['ZA0 0 coal', 'ZA0 0 onwind', 'ZA0 0 ror', 'ZA0 0 solar', 'ZA0 1 CCGT', 'ZA0 1 offwind-ac', 'ZA0 1 offwind-dc', 'ZA0 1 onwind', 'ZA0 1 solar', 'ZA0 2 onwind', 'ZA0 2 ror', 'ZA0 2 solar', 'ZA0 3 biomass', 'ZA0 3 coal', 'ZA0 3 onwind', 'ZA0 3 ror', 'ZA0 3 solar', 'ZA0 4 CCGT', 'ZA0 4 coal', 'ZA0 4 onwind', 'ZA0 4 solar', 'ZA0 5 CCGT', 'ZA0 5 nuclear', 'ZA0 5 offwind-ac', 'ZA0 5 offwind-dc', 'ZA0 5 onwind', 'ZA0 5 solar', 'ZA0 6 onwind', 'ZA0 6 solar', 'ZA0 7 CCGT', 'ZA0 7 coal', 'ZA0 7 offwind-ac', 'ZA0 7 offw

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'ZA0 0 PHS'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/tmp/ipython-input-2715216793.py", line 23, in quick_test
    obs, reward, terminated, truncated, info = env.step(action)
                                               ^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1889143200.py", line 835, in step
    self.network.gener

In [ ]:
# Can add these to the base env to debug
# # Debug function to identify the storage unit issue
# def debug_storage_units(env_instance):
#     """
#     Debug function to check storage unit setup and identify issues
#     """
#     print("=== Storage Unit Debug ===")

#     network = env_instance.network

#     # Check if storage units exist
#     print(f"Number of storage units: {len(network.storage_units)}")
#     if len(network.storage_units) > 0:
#         print(f"Storage unit names: {list(network.storage_units.index)}")
#         print(f"Storage unit columns: {list(network.storage_units.columns)}")

#     # Check environment storage attributes
#     print(f"env.n_storage: {env_instance.n_storage}")
#     print(f"env.storage_names: {env_instance.storage_names}")

#     # Check if storage_units_t exists
#     print(f"Has storage_units_t: {hasattr(network, 'storage_units_t')}")

#     if hasattr(network, 'storage_units_t'):
#         # Check what attributes exist in storage_units_t
#         storage_t_attrs = [attr for attr in dir(network.storage_units_t) if not attr.startswith('_')]
#         print(f"storage_units_t attributes: {storage_t_attrs}")

#         # Check specific DataFrames
#         required_attrs = ['p_set', 'p_dispatch', 'p_store', 'state_of_charge']
#         for attr in required_attrs:
#             if hasattr(network.storage_units_t, attr):
#                 df = getattr(network.storage_units_t, attr)
#                 print(f"storage_units_t.{attr}: shape={df.shape}, columns={list(df.columns)}")
#             else:
#                 print(f"storage_units_t.{attr}: MISSING")

#     # Check action space structure
#     print(f"\nAction space shape: {env_instance.action_space.shape}")
#     print(f"Action structure: {env_instance.action_structure}")


# def debug_step_function(env_instance):
#     """
#     Debug a single step to see where it fails
#     """
#     print("\n=== Step Function Debug ===")

#     try:
#         # Reset environment
#         print("1. Resetting environment...")
#         obs, info = env_instance.reset(seed=42)
#         print("   ✓ Reset successful")

#         # Sample action
#         print("2. Sampling action...")
#         action = env_instance.action_space.sample()
#         print(f"   ✓ Action sampled: shape={action.shape}")

#         # Scale action
#         print("3. Scaling action...")
#         scaled_actions = env_instance.scale_action(action)
#         print("   ✓ Action scaled successfully")
#         for component, values in scaled_actions.items():
#             print(f"     {component}: {len(values)} values")

#         # Try to apply actions manually to see where it fails
#         print("4. Applying actions manually...")

#         # Dispatchable generators
#         if env_instance.n_dispatchable > 0:
#             print("   4a. Applying dispatchable generator actions...")
#             for i, gen_name in enumerate(env_instance.dispatchable_names):
#                 try:
#                     env_instance.network.generators_t.p_set.iloc[env_instance.snapshot_idx,
#                         env_instance.network.generators_t.p_set.columns.get_loc(gen_name)] = scaled_actions['dispatchable'][i]
#                     print(f"      ✓ Applied to {gen_name}")
#                 except Exception as e:
#                     print(f"      ✗ Failed on {gen_name}: {e}")
#                     return False

#         # Renewable generators
#         if env_instance.n_renewable > 0:
#             print("   4b. Applying renewable generator actions...")
#             for i, gen_name in enumerate(env_instance.renewable_names):
#                 try:
#                     env_instance.network.generators_t.p_set.iloc[env_instance.snapshot_idx,
#                         env_instance.network.generators_t.p_set.columns.get_loc(gen_name)] = scaled_actions['renewable'][i]
#                     print(f"      ✓ Applied to {gen_name}")
#                 except Exception as e:
#                     print(f"      ✗ Failed on {gen_name}: {e}")
#                     return False

#         # Storage units - this is likely where it fails
#         if env_instance.n_storage > 0:
#             print("   4c. Applying storage unit actions...")
#             for i, storage_name in enumerate(env_instance.storage_names):
#                 print(f"      Trying storage unit: {storage_name}")

#                 try:
#                     # Check if the DataFrames exist first
#                     if not hasattr(env_instance.network.storage_units_t, 'p_set'):
#                         print(f"      ✗ storage_units_t.p_set doesn't exist!")
#                         return False

#                     # Try p_set
#                     env_instance.network.storage_units_t.p_set.iloc[env_instance.snapshot_idx,
#                         env_instance.network.storage_units_t.p_set.columns.get_loc(storage_name)] = scaled_actions['storage_p_set'][i]
#                     print(f"      ✓ Applied p_set to {storage_name}")

#                     # Try p_dispatch
#                     env_instance.network.storage_units_t.p_dispatch.iloc[env_instance.snapshot_idx,
#                         env_instance.network.storage_units_t.p_dispatch.columns.get_loc(storage_name)] = scaled_actions['storage_p_dispatch'][i]
#                     print(f"      ✓ Applied p_dispatch to {storage_name}")

#                     # Try p_store
#                     env_instance.network.storage_units_t.p_store.iloc[env_instance.snapshot_idx,
#                         env_instance.network.storage_units_t.p_store.columns.get_loc(storage_name)] = scaled_actions['storage_p_dispatch'][i] - scaled_actions['storage_p_set'][i]
#                     print(f"      ✓ Applied p_store to {storage_name}")

#                 except Exception as e:
#                     print(f"      ✗ Failed on storage {storage_name}: {e}")
#                     print(f"        Error type: {type(e).__name__}")
#                     return False

#         print("5. All actions applied successfully!")
#         return True

#     except Exception as e:
#         print(f"Debug failed at step: {e}")
#         import traceback
#         traceback.print_exc()
#         return False

In [ ]:
# # Simple test to run immediately
# def quick_test():
#     """Quick test you can run to check basic functionality"""
#     try:
#         # Update with your network file path
#         network_file = '/content/drive/MyDrive/Colab_Notebooks/networks_1_week_connected/elec_s_10_ec_lc1.0_1h.nc'


#         print("Creating environment...")
#         env = Env2Gen1LoadConstr(network_file=network_file, episode_length=3)

#         print("Validating network structure...")
#         env.validate_network_structure()

#         print("Testing reset...")
#         obs, info = env.reset()
#         print(f"Reset successful - observation shape: {obs.shape}")

#         print("Testing single step...")
#         action = env.action_space.sample()
#         env.debug_action_scaling(action)

#         obs, reward, terminated, truncated, info = env.step(action)
#         print(f"Step successful - reward: {reward:.4f}")
#         print(f"Constraints satisfied: {info.get('constraints_satisfied', 'N/A')}")

#         print("✓ Quick test passed!")

#     except Exception as e:
#         print(f"✗ Quick test failed: {e}")
#         import traceback
#         traceback.print_exc()

In [52]:
# Construct the full path to the network file in Google Drive
network_file_path = '/content/drive/MyDrive/Colab_Notebooks/networks_1_year_connected/elec_s_10_ec_lc1.0_1h.nc'

# Check if the file exists before attempting to create the environment
if not os.path.exists(network_file_path):
    print(f"Error: Network file not found at {network_file_path}")
else:
    env = Env2Gen1LoadConstr(network_file=network_file_path)

Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA0 5', 'ZA0 6', 'ZA0 7',
       'ZA0 8', 'ZA0 9', 'ZA1 0', 'ZA2 0', 'ZA3 0'],
      dtype='object', name='Bus')
Index(['0', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6',
       '7', '8', '9', 'lines new ZA0 4 <-> ZA2 0 AC',
       'lines new ZA0 0 <-> ZA1 0 AC', 'lines new ZA0 0 <-> ZA3 0 AC'],
      dtype='object', name='Line')
/usr/local/lib/python3.12/dist-packages/linopy/expressions.py:1861: FutureWarning:

In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): '_term' ('_term',) The recommendation is to set join explicitly for this case.



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded variable mapping from: /content/drive/MyDrive/Colab_Notebooks/var_constraint_map/elec_s_10_ec_lc1.0_1h_var_id_to_name.pkl
Loaded constraints from: /content/drive/MyDrive/Colab_Notebooks/var_constraint_map/elec_s_10_ec_lc1.0_1h_constraints.pkl


In [55]:
env.network.storage_units_t

{'p_min_pu': Empty DataFrame
 Columns: []
 Index: [2013-01-01 00:00:00, 2013-01-01 01:00:00, 2013-01-01 02:00:00, 2013-01-01 03:00:00, 2013-01-01 04:00:00, 2013-01-01 05:00:00, 2013-01-01 06:00:00, 2013-01-01 07:00:00, 2013-01-01 08:00:00, 2013-01-01 09:00:00, 2013-01-01 10:00:00, 2013-01-01 11:00:00, 2013-01-01 12:00:00, 2013-01-01 13:00:00, 2013-01-01 14:00:00, 2013-01-01 15:00:00, 2013-01-01 16:00:00, 2013-01-01 17:00:00, 2013-01-01 18:00:00, 2013-01-01 19:00:00, 2013-01-01 20:00:00, 2013-01-01 21:00:00, 2013-01-01 22:00:00, 2013-01-01 23:00:00, 2013-01-02 00:00:00, 2013-01-02 01:00:00, 2013-01-02 02:00:00, 2013-01-02 03:00:00, 2013-01-02 04:00:00, 2013-01-02 05:00:00, 2013-01-02 06:00:00, 2013-01-02 07:00:00, 2013-01-02 08:00:00, 2013-01-02 09:00:00, 2013-01-02 10:00:00, 2013-01-02 11:00:00, 2013-01-02 12:00:00, 2013-01-02 13:00:00, 2013-01-02 14:00:00, 2013-01-02 15:00:00, 2013-01-02 16:00:00, 2013-01-02 17:00:00, 2013-01-02 18:00:00, 2013-01-02 19:00:00, 2013-01-02 20:00:00, 2013

In [ ]:
network= pypsa.Network('/content/drive/MyDrive/Colab_Notebooks/networks_1_week_connected/elec_s_10_ec_lc1.0_1h.nc')


In [ ]:
soc_max = network.storage_units.p_nom * network.storage_units.max_hours
print(soc_max)

StorageUnit
ZA0 0 PHS      0.000000e+00
ZA0 5 PHS      0.000000e+00
ZA0 6 hydro    2.298976e+06
dtype: float64


In [ ]:
print(network.storage_units_t.spill)

Empty DataFrame
Columns: []
Index: [2013-01-01 00:00:00, 2013-01-01 01:00:00, 2013-01-01 02:00:00, 2013-01-01 03:00:00, 2013-01-01 04:00:00, 2013-01-01 05:00:00, 2013-01-01 06:00:00, 2013-01-01 07:00:00, 2013-01-01 08:00:00, 2013-01-01 09:00:00, 2013-01-01 10:00:00, 2013-01-01 11:00:00, 2013-01-01 12:00:00, 2013-01-01 13:00:00, 2013-01-01 14:00:00, 2013-01-01 15:00:00, 2013-01-01 16:00:00, 2013-01-01 17:00:00, 2013-01-01 18:00:00, 2013-01-01 19:00:00, 2013-01-01 20:00:00, 2013-01-01 21:00:00, 2013-01-01 22:00:00, 2013-01-01 23:00:00, 2013-01-02 00:00:00, 2013-01-02 01:00:00, 2013-01-02 02:00:00, 2013-01-02 03:00:00, 2013-01-02 04:00:00, 2013-01-02 05:00:00, 2013-01-02 06:00:00, 2013-01-02 07:00:00, 2013-01-02 08:00:00, 2013-01-02 09:00:00, 2013-01-02 10:00:00, 2013-01-02 11:00:00, 2013-01-02 12:00:00, 2013-01-02 13:00:00, 2013-01-02 14:00:00, 2013-01-02 15:00:00, 2013-01-02 16:00:00, 2013-01-02 17:00:00, 2013-01-02 18:00:00, 2013-01-02 19:00:00, 2013-01-02 20:00:00, 2013-01-02 21:00:00

In [ ]:
print(network.controllable_one_port_components)

{'Generator', 'Store', 'StorageUnit', 'Load'}


In [ ]:
print(network.controllable_branch_components)

{'Link'}


In [ ]:
print(network.links_t)

{'efficiency': Empty DataFrame
Columns: []
Index: [2013-01-01 00:00:00, 2013-01-01 01:00:00, 2013-01-01 02:00:00, 2013-01-01 03:00:00, 2013-01-01 04:00:00, 2013-01-01 05:00:00, 2013-01-01 06:00:00, 2013-01-01 07:00:00, 2013-01-01 08:00:00, 2013-01-01 09:00:00, 2013-01-01 10:00:00, 2013-01-01 11:00:00, 2013-01-01 12:00:00, 2013-01-01 13:00:00, 2013-01-01 14:00:00, 2013-01-01 15:00:00, 2013-01-01 16:00:00, 2013-01-01 17:00:00, 2013-01-01 18:00:00, 2013-01-01 19:00:00, 2013-01-01 20:00:00, 2013-01-01 21:00:00, 2013-01-01 22:00:00, 2013-01-01 23:00:00, 2013-01-02 00:00:00, 2013-01-02 01:00:00, 2013-01-02 02:00:00, 2013-01-02 03:00:00, 2013-01-02 04:00:00, 2013-01-02 05:00:00, 2013-01-02 06:00:00, 2013-01-02 07:00:00, 2013-01-02 08:00:00, 2013-01-02 09:00:00, 2013-01-02 10:00:00, 2013-01-02 11:00:00, 2013-01-02 12:00:00, 2013-01-02 13:00:00, 2013-01-02 14:00:00, 2013-01-02 15:00:00, 2013-01-02 16:00:00, 2013-01-02 17:00:00, 2013-01-02 18:00:00, 2013-01-02 19:00:00, 2013-01-02 20:00:00, 2013

In [ ]:
print(network.controllable_branches)

<bound method Network.controllable_branches of PyPSA Network ''
----------------
Components:
 - Bus: 13
 - Carrier: 15
 - Generator: 55
 - GlobalConstraint: 1
 - Line: 19
 - Load: 13
 - StorageUnit: 3
Snapshots: 8760>


In [ ]:
# Example usage
for storage_name in network.storage_units.index:
    # print(f"Storage unit: {storage_name}")
    # storage_unit = network.storage_units.loc[storage_name]
    # eff_dispatch = storage_unit.efficiency_dispatch
    # print(eff_dispatch)
    # eff_store = storage_unit.get('efficiency_store', 1.0)
    # standing_loss = storage_unit.get('standing_loss', 0.0)
    print(network.storage_units.loc[storage_name].max_hours)


0.0
0.0
3831.6270020496813


In [ ]:
print(network.storage_units.marginal_cost)

Empty DataFrame
Columns: []
Index: [2013-01-01 00:00:00, 2013-01-01 01:00:00, 2013-01-01 02:00:00, 2013-01-01 03:00:00, 2013-01-01 04:00:00, 2013-01-01 05:00:00, 2013-01-01 06:00:00, 2013-01-01 07:00:00, 2013-01-01 08:00:00, 2013-01-01 09:00:00, 2013-01-01 10:00:00, 2013-01-01 11:00:00, 2013-01-01 12:00:00, 2013-01-01 13:00:00, 2013-01-01 14:00:00, 2013-01-01 15:00:00, 2013-01-01 16:00:00, 2013-01-01 17:00:00, 2013-01-01 18:00:00, 2013-01-01 19:00:00, 2013-01-01 20:00:00, 2013-01-01 21:00:00, 2013-01-01 22:00:00, 2013-01-01 23:00:00, 2013-01-02 00:00:00, 2013-01-02 01:00:00, 2013-01-02 02:00:00, 2013-01-02 03:00:00, 2013-01-02 04:00:00, 2013-01-02 05:00:00, 2013-01-02 06:00:00, 2013-01-02 07:00:00, 2013-01-02 08:00:00, 2013-01-02 09:00:00, 2013-01-02 10:00:00, 2013-01-02 11:00:00, 2013-01-02 12:00:00, 2013-01-02 13:00:00, 2013-01-02 14:00:00, 2013-01-02 15:00:00, 2013-01-02 16:00:00, 2013-01-02 17:00:00, 2013-01-02 18:00:00, 2013-01-02 19:00:00, 2013-01-02 20:00:00, 2013-01-02 21:00:00

In [ ]:
print(network.storage_units.marginal_cost_storage)

Empty DataFrame
Columns: []
Index: [2013-01-01 00:00:00, 2013-01-01 01:00:00, 2013-01-01 02:00:00, 2013-01-01 03:00:00, 2013-01-01 04:00:00, 2013-01-01 05:00:00, 2013-01-01 06:00:00, 2013-01-01 07:00:00, 2013-01-01 08:00:00, 2013-01-01 09:00:00, 2013-01-01 10:00:00, 2013-01-01 11:00:00, 2013-01-01 12:00:00, 2013-01-01 13:00:00, 2013-01-01 14:00:00, 2013-01-01 15:00:00, 2013-01-01 16:00:00, 2013-01-01 17:00:00, 2013-01-01 18:00:00, 2013-01-01 19:00:00, 2013-01-01 20:00:00, 2013-01-01 21:00:00, 2013-01-01 22:00:00, 2013-01-01 23:00:00, 2013-01-02 00:00:00, 2013-01-02 01:00:00, 2013-01-02 02:00:00, 2013-01-02 03:00:00, 2013-01-02 04:00:00, 2013-01-02 05:00:00, 2013-01-02 06:00:00, 2013-01-02 07:00:00, 2013-01-02 08:00:00, 2013-01-02 09:00:00, 2013-01-02 10:00:00, 2013-01-02 11:00:00, 2013-01-02 12:00:00, 2013-01-02 13:00:00, 2013-01-02 14:00:00, 2013-01-02 15:00:00, 2013-01-02 16:00:00, 2013-01-02 17:00:00, 2013-01-02 18:00:00, 2013-01-02 19:00:00, 2013-01-02 20:00:00, 2013-01-02 21:00:00

In [ ]:
print(network.storage_units.marginal_cost_quadratic)

Empty DataFrame
Columns: []
Index: [2013-01-01 00:00:00, 2013-01-01 01:00:00, 2013-01-01 02:00:00, 2013-01-01 03:00:00, 2013-01-01 04:00:00, 2013-01-01 05:00:00, 2013-01-01 06:00:00, 2013-01-01 07:00:00, 2013-01-01 08:00:00, 2013-01-01 09:00:00, 2013-01-01 10:00:00, 2013-01-01 11:00:00, 2013-01-01 12:00:00, 2013-01-01 13:00:00, 2013-01-01 14:00:00, 2013-01-01 15:00:00, 2013-01-01 16:00:00, 2013-01-01 17:00:00, 2013-01-01 18:00:00, 2013-01-01 19:00:00, 2013-01-01 20:00:00, 2013-01-01 21:00:00, 2013-01-01 22:00:00, 2013-01-01 23:00:00, 2013-01-02 00:00:00, 2013-01-02 01:00:00, 2013-01-02 02:00:00, 2013-01-02 03:00:00, 2013-01-02 04:00:00, 2013-01-02 05:00:00, 2013-01-02 06:00:00, 2013-01-02 07:00:00, 2013-01-02 08:00:00, 2013-01-02 09:00:00, 2013-01-02 10:00:00, 2013-01-02 11:00:00, 2013-01-02 12:00:00, 2013-01-02 13:00:00, 2013-01-02 14:00:00, 2013-01-02 15:00:00, 2013-01-02 16:00:00, 2013-01-02 17:00:00, 2013-01-02 18:00:00, 2013-01-02 19:00:00, 2013-01-02 20:00:00, 2013-01-02 21:00:00

In [ ]:
print(network.storage_units.spill_cost)

Empty DataFrame
Columns: []
Index: [2013-01-01 00:00:00, 2013-01-01 01:00:00, 2013-01-01 02:00:00, 2013-01-01 03:00:00, 2013-01-01 04:00:00, 2013-01-01 05:00:00, 2013-01-01 06:00:00, 2013-01-01 07:00:00, 2013-01-01 08:00:00, 2013-01-01 09:00:00, 2013-01-01 10:00:00, 2013-01-01 11:00:00, 2013-01-01 12:00:00, 2013-01-01 13:00:00, 2013-01-01 14:00:00, 2013-01-01 15:00:00, 2013-01-01 16:00:00, 2013-01-01 17:00:00, 2013-01-01 18:00:00, 2013-01-01 19:00:00, 2013-01-01 20:00:00, 2013-01-01 21:00:00, 2013-01-01 22:00:00, 2013-01-01 23:00:00, 2013-01-02 00:00:00, 2013-01-02 01:00:00, 2013-01-02 02:00:00, 2013-01-02 03:00:00, 2013-01-02 04:00:00, 2013-01-02 05:00:00, 2013-01-02 06:00:00, 2013-01-02 07:00:00, 2013-01-02 08:00:00, 2013-01-02 09:00:00, 2013-01-02 10:00:00, 2013-01-02 11:00:00, 2013-01-02 12:00:00, 2013-01-02 13:00:00, 2013-01-02 14:00:00, 2013-01-02 15:00:00, 2013-01-02 16:00:00, 2013-01-02 17:00:00, 2013-01-02 18:00:00, 2013-01-02 19:00:00, 2013-01-02 20:00:00, 2013-01-02 21:00:00

In [ ]:
for attr in dir(network.storage_units):
  if "cost" in attr:
    print(attr)
    #print(network.storage_units.get(attr))

capital_cost
marginal_cost
marginal_cost_quadratic
marginal_cost_storage
spill_cost


In [ ]:
print(network.generators_t.marginal_cost.equals(network.generators.marginal_cost))

False


In [ ]:
for attr in dir(network.generators):
  if "cost" in attr:
    print(attr)
    #print(network.generators.get(attr))

capital_cost
marginal_cost
marginal_cost_quadratic
shut_down_cost
stand_by_cost
start_up_cost


In [ ]:
print(network.loads_t.p_set)

Load                       ZA0 0        ZA0 1        ZA0 2        ZA0 3  \
snapshot                                                                  
2013-01-01 00:00:00  2238.549587  1801.350559  1125.414339  4021.337885   
2013-01-01 01:00:00  2219.249560  1785.819916  1115.711392  3986.667252   
2013-01-01 02:00:00  2219.249560  1785.819916  1115.711392  3986.667252   
2013-01-01 03:00:00  2285.746017  1839.329309  1149.141997  4106.121708   
2013-01-01 04:00:00  2385.624702  1919.701228  1199.355271  4285.544108   
...                          ...          ...          ...          ...   
2013-12-31 19:00:00  2962.091024  2383.581026  1489.169475  5321.110117   
2013-12-31 20:00:00  2833.830222  2280.370149  1424.687300  5090.701988   
2013-12-31 21:00:00  2733.463476  2199.605490  1374.228657  4910.402834   
2013-12-31 22:00:00  2455.916369  1976.264609  1234.693890  4411.816292   
2013-12-31 23:00:00  2337.085769  1880.642172  1174.952680  4198.348609   

Load                    

In [ ]:
print(network.storage_units_t.state_of_charge)

Empty DataFrame
Columns: []
Index: [2013-01-01 00:00:00, 2013-01-01 01:00:00, 2013-01-01 02:00:00, 2013-01-01 03:00:00, 2013-01-01 04:00:00, 2013-01-01 05:00:00, 2013-01-01 06:00:00, 2013-01-01 07:00:00, 2013-01-01 08:00:00, 2013-01-01 09:00:00, 2013-01-01 10:00:00, 2013-01-01 11:00:00, 2013-01-01 12:00:00, 2013-01-01 13:00:00, 2013-01-01 14:00:00, 2013-01-01 15:00:00, 2013-01-01 16:00:00, 2013-01-01 17:00:00, 2013-01-01 18:00:00, 2013-01-01 19:00:00, 2013-01-01 20:00:00, 2013-01-01 21:00:00, 2013-01-01 22:00:00, 2013-01-01 23:00:00, 2013-01-02 00:00:00, 2013-01-02 01:00:00, 2013-01-02 02:00:00, 2013-01-02 03:00:00, 2013-01-02 04:00:00, 2013-01-02 05:00:00, 2013-01-02 06:00:00, 2013-01-02 07:00:00, 2013-01-02 08:00:00, 2013-01-02 09:00:00, 2013-01-02 10:00:00, 2013-01-02 11:00:00, 2013-01-02 12:00:00, 2013-01-02 13:00:00, 2013-01-02 14:00:00, 2013-01-02 15:00:00, 2013-01-02 16:00:00, 2013-01-02 17:00:00, 2013-01-02 18:00:00, 2013-01-02 19:00:00, 2013-01-02 20:00:00, 2013-01-02 21:00:00

In [ ]:
for storage_name in network.storage_units.index:
  # Use .loc for direct assignment to avoid SettingWithCopyWarning
  network.storage_units.loc[storage_name, 'cyclic_state_of_charge'] = False

In [ ]:
print(network.storage_units.cyclic_state_of_charge)

StorageUnit
ZA0 0 PHS      False
ZA0 5 PHS      False
ZA0 6 hydro    False
Name: cyclic_state_of_charge, dtype: bool


In [ ]:
print(network.storage_units.state_of_charge_initial)

StorageUnit
ZA0 0 PHS      0.0
ZA0 5 PHS      0.0
ZA0 6 hydro    0.0
Name: state_of_charge_initial, dtype: float64


In [ ]:
print(network.storage_units.efficiency_store)

StorageUnit
ZA0 0 PHS      0.866025
ZA0 5 PHS      0.866025
ZA0 6 hydro    0.000000
Name: efficiency_store, dtype: float64


In [ ]:
print(network.storage_units.efficiency_dispatch)

StorageUnit
ZA0 0 PHS      0.866025
ZA0 5 PHS      0.866025
ZA0 6 hydro    0.900000
Name: efficiency_dispatch, dtype: float64


In [ ]:
print(network.storage_units_t.inflow)

StorageUnit          ZA0 6 hydro
snapshot                        
2013-01-01 00:00:00    37.058718
2013-01-01 01:00:00    25.498610
2013-01-01 02:00:00    34.095640
2013-01-01 03:00:00    45.030080
2013-01-01 04:00:00   186.270643
...                          ...
2013-12-31 19:00:00   330.134209
2013-12-31 20:00:00   202.406836
2013-12-31 21:00:00   203.539523
2013-12-31 22:00:00    92.002816
2013-12-31 23:00:00    49.727819

[8760 rows x 1 columns]


In [ ]:
#After import
print(network.generators)

                    bus control type         p_nom  p_nom_mod  \
Generator                                                       
ZA0 0 coal        ZA0 0   Slack        5743.000000        0.0   
ZA0 0 onwind      ZA0 0                 102.058491        0.0   
ZA0 0 ror         ZA0 0                   7.000000        0.0   
ZA0 0 solar       ZA0 0                 247.208836        0.0   
ZA0 1 CCGT        ZA0 1                 740.000000        0.0   
ZA0 1 offwind-ac  ZA0 1                   0.000000        0.0   
ZA0 1 offwind-dc  ZA0 1                   0.000000        0.0   
ZA0 1 onwind      ZA0 1                 286.542875        0.0   
ZA0 1 solar       ZA0 1                 413.147221        0.0   
ZA0 2 onwind      ZA0 2                 929.632819        0.0   
ZA0 2 ror         ZA0 2                  12.570000        0.0   
ZA0 2 solar       ZA0 2                1276.694834        0.0   
ZA0 3 biomass     ZA0 3                  10.100000        0.0   
ZA0 3 coal        ZA0 3  

In [ ]:
slack_gens = network.generators[network.generators.control == 'Slack']
slack_buses = slack_gens.bus.unique()
print("Slack buses (from control attribute):")
print(slack_buses)
print("Slack gens: ",slack_gens )

Slack buses (from control attribute):
['ZA0 0' 'ZA1 0' 'ZA2 0' 'ZA3 0']
Slack gens:                      bus control type        p_nom  p_nom_mod  \
Generator                                                      
ZA0 0 coal        ZA0 0   Slack       5743.000000        0.0   
ZA1 0 offwind-ac  ZA1 0   Slack          0.000000        0.0   
ZA2 0 onwind      ZA2 0   Slack          1.981033        0.0   
ZA3 0 offwind-ac  ZA3 0   Slack          0.000000        0.0   

                  p_nom_extendable    p_nom_min  p_nom_max  p_min_pu  \
Generator                                                              
ZA0 0 coal                   False  5743.000000        inf       0.0   
ZA1 0 offwind-ac             False     0.000000   0.000000       0.0   
ZA2 0 onwind                 False     1.981033   1.981033       0.0   
ZA3 0 offwind-ac             False     0.000000   0.000000       0.0   

                  p_max_pu  ...  min_up_time  min_down_time  up_time_before  \
Generator         

In [ ]:
generators_with_p_max_pu_t = network.generators_t.p_max_pu.columns
        # Filter network.generators to find those NOT in the list above. These are the dispatchable generators
generators_without_p_max_pu_t = network.generators[~network.generators.index.isin(generators_with_p_max_pu_t)].index
print(network.generators[(network.generators.control == "Slack")].index)

Index(['ZA0 0 coal', 'ZA1 0 offwind-ac', 'ZA2 0 onwind', 'ZA3 0 offwind-ac'], dtype='object', name='Generator')


In [ ]:
print("=== NON-EMPTY GENERATORS_T ATTRIBUTES ===")
for attr_name in dir(network.generators_t):
    if not attr_name.startswith('_'):  # Skip private attributes
        attr = getattr(network.generators_t, attr_name)
        if hasattr(attr, 'columns') and hasattr(attr, 'empty'):
            if not attr.empty:
                print(f"{attr_name}: {len(attr.columns)} generators, {len(attr.index)} timesteps")
                print(f"  Generators: {attr.columns.tolist()}")

=== NON-EMPTY GENERATORS_T ATTRIBUTES ===
p_max_pu: 42 generators, 8760 timesteps
  Generators: ['ZA0 0 onwind', 'ZA0 0 ror', 'ZA0 0 solar', 'ZA0 1 offwind-ac', 'ZA0 1 offwind-dc', 'ZA0 1 onwind', 'ZA0 1 solar', 'ZA0 2 onwind', 'ZA0 2 ror', 'ZA0 2 solar', 'ZA0 3 onwind', 'ZA0 3 ror', 'ZA0 3 solar', 'ZA0 4 onwind', 'ZA0 4 solar', 'ZA0 5 offwind-ac', 'ZA0 5 offwind-dc', 'ZA0 5 onwind', 'ZA0 5 solar', 'ZA0 6 onwind', 'ZA0 6 solar', 'ZA0 7 offwind-ac', 'ZA0 7 offwind-dc', 'ZA0 7 onwind', 'ZA0 7 solar', 'ZA0 8 offwind-ac', 'ZA0 8 offwind-dc', 'ZA0 8 onwind', 'ZA0 8 ror', 'ZA0 8 solar', 'ZA0 9 onwind', 'ZA0 9 solar', 'ZA1 0 offwind-ac', 'ZA1 0 offwind-dc', 'ZA1 0 onwind', 'ZA1 0 solar', 'ZA2 0 onwind', 'ZA2 0 solar', 'ZA3 0 offwind-ac', 'ZA3 0 offwind-dc', 'ZA3 0 onwind', 'ZA3 0 solar']


In [ ]:
network.optimize()
p_after_optimize=network.generators_t.p
print(p_after_optimize)
network_after_optimize=network

Index(['0', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6',
       '7', '8', '9', 'lines new ZA0 4 <-> ZA2 0 AC',
       'lines new ZA0 0 <-> ZA1 0 AC', 'lines new ZA0 0 <-> ZA3 0 AC'],
      dtype='object', name='Line')
Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA0 5', 'ZA0 6', 'ZA0 7',
       'ZA0 8', 'ZA0 9', 'ZA1 0', 'ZA2 0', 'ZA3 0'],
      dtype='object', name='Bus')
Writing continuous variables.: 100%|██████████| 7/7 [00:01<00:00,  5.45it/s]


Generator            ZA0 0 coal  ZA0 0 onwind  ZA0 0 ror  ZA0 0 solar  \
snapshot                                                                
2013-01-01 00:00:00        -0.0      2.006375   2.228640         -0.0   
2013-01-01 01:00:00      5743.0      0.973489   2.896928         -0.0   
2013-01-01 02:00:00      5743.0      0.655491   3.742913         -0.0   
2013-01-01 03:00:00      5743.0      0.427377   2.088971         -0.0   
2013-01-01 04:00:00        -0.0      0.135659   2.424019         -0.0   
...                         ...           ...        ...          ...   
2013-12-31 19:00:00      5743.0     27.543664   0.331752         -0.0   
2013-12-31 20:00:00        -0.0     24.450197   0.374511         -0.0   
2013-12-31 21:00:00      5743.0     19.053085   0.432341         -0.0   
2013-12-31 22:00:00      5743.0     13.565647   0.393768         -0.0   
2013-12-31 23:00:00      5743.0      9.425721   0.370420         -0.0   

Generator            ZA0 1 CCGT  ZA0 1 offwind-ac 

In [ ]:
print("=== NON-EMPTY GENERATORS_T ATTRIBUTES ===")
for attr_name in dir(network_after_optimize.generators_t):
    if not attr_name.startswith('_'):  # Skip private attributes
        attr = getattr(network_after_optimize.generators_t, attr_name)
        if hasattr(attr, 'columns') and hasattr(attr, 'empty'):
            if not attr.empty:
                print(f"{attr_name}: {len(attr.columns)} generators, {len(attr.index)} timesteps")
                print(f"  Generators: {attr.columns.tolist()}")

=== NON-EMPTY GENERATORS_T ATTRIBUTES ===
p: 55 generators, 8760 timesteps
  Generators: ['ZA0 0 coal', 'ZA0 0 onwind', 'ZA0 0 ror', 'ZA0 0 solar', 'ZA0 1 CCGT', 'ZA0 1 offwind-ac', 'ZA0 1 offwind-dc', 'ZA0 1 onwind', 'ZA0 1 solar', 'ZA0 2 onwind', 'ZA0 2 ror', 'ZA0 2 solar', 'ZA0 3 biomass', 'ZA0 3 coal', 'ZA0 3 onwind', 'ZA0 3 ror', 'ZA0 3 solar', 'ZA0 4 CCGT', 'ZA0 4 coal', 'ZA0 4 onwind', 'ZA0 4 solar', 'ZA0 5 CCGT', 'ZA0 5 nuclear', 'ZA0 5 offwind-ac', 'ZA0 5 offwind-dc', 'ZA0 5 onwind', 'ZA0 5 solar', 'ZA0 6 onwind', 'ZA0 6 solar', 'ZA0 7 CCGT', 'ZA0 7 coal', 'ZA0 7 offwind-ac', 'ZA0 7 offwind-dc', 'ZA0 7 onwind', 'ZA0 7 solar', 'ZA0 8 CCGT', 'ZA0 8 biomass', 'ZA0 8 offwind-ac', 'ZA0 8 offwind-dc', 'ZA0 8 onwind', 'ZA0 8 ror', 'ZA0 8 solar', 'ZA0 9 coal', 'ZA0 9 onwind', 'ZA0 9 solar', 'ZA1 0 offwind-ac', 'ZA1 0 offwind-dc', 'ZA1 0 onwind', 'ZA1 0 solar', 'ZA2 0 onwind', 'ZA2 0 solar', 'ZA3 0 offwind-ac', 'ZA3 0 offwind-dc', 'ZA3 0 onwind', 'ZA3 0 solar']
p_max_pu: 42 generators,

In [ ]:
network= pypsa.Network('/content/drive/MyDrive/Colab_Notebooks/networks_1_week_connected/elec_s_10_ec_lc1.0_1h.nc')
#print(network.generators.carrier.unique())
#print(network.generators_t)
#print(getattr(network.generators_t, 'p_max_pu'))

# Get the list of generators with non-empty p_max_pu
generators_with_p_max_pu = network.generators_t.p_max_pu.columns
#print(generators_with_p_max_pu)
# Filter network.generators to find those NOT in the list above
generators_without_p_max_pu_timeseries = network.generators[
    ~network.generators.index.isin(generators_with_p_max_pu)
]


print(generators_without_p_max_pu_timeseries)
# p_before_optimize=network.generators_t.p
# print(p_before_optimize)

                 bus control type         p_nom  p_nom_mod  p_nom_extendable  \
Generator                                                                      
ZA0 0 coal     ZA0 0   Slack        5743.000000        0.0             False   
ZA0 1 CCGT     ZA0 1                 740.000000        0.0             False   
ZA0 3 biomass  ZA0 3                  10.100000        0.0             False   
ZA0 3 coal     ZA0 3                2352.000000        0.0             False   
ZA0 4 CCGT     ZA0 4                 301.740341        0.0             False   
ZA0 4 coal     ZA0 4               38589.962174        0.0             False   
ZA0 5 CCGT     ZA0 5                1338.300000        0.0             False   
ZA0 5 nuclear  ZA0 5                1940.000000        0.0             False   
ZA0 7 CCGT     ZA0 7                 670.000000        0.0             False   
ZA0 7 coal     ZA0 7                  72.300000        0.0             False   
ZA0 8 CCGT     ZA0 8                 335

In [ ]:
# Get the list of generators with non-empty p_max_pu. These are the non-dispatchable.
generators_with_p_max_pu_t = network.generators_t.p_max_pu.columns
# Filter network.generators to find those NOT in the list above
generators_without_p_max_pu_t = network.generators[
    ~network.generators.index.isin(generators_with_p_max_pu_t)
].index

slack_generator_idx = network.generators[
    (network.generators.control == "Slack") & (~network.generators.index.isin(generators_with_p_max_pu_t))
].index

non_slack_gens = network.generators[
    (network.generators.control != "Slack") & (~network.generators.index.isin(generators_with_p_max_pu_t))
].index

print(non_slack_gens)

Index(['ZA0 1 CCGT', 'ZA0 3 biomass', 'ZA0 3 coal', 'ZA0 4 CCGT', 'ZA0 4 coal',
       'ZA0 5 CCGT', 'ZA0 5 nuclear', 'ZA0 7 CCGT', 'ZA0 7 coal', 'ZA0 8 CCGT',
       'ZA0 8 biomass', 'ZA0 9 coal'],
      dtype='object', name='Generator')


In [ ]:
print(network.generators_t.p_set)

Empty DataFrame
Columns: []
Index: [2013-01-01 00:00:00, 2013-01-01 01:00:00, 2013-01-01 02:00:00, 2013-01-01 03:00:00, 2013-01-01 04:00:00, 2013-01-01 05:00:00, 2013-01-01 06:00:00, 2013-01-01 07:00:00, 2013-01-01 08:00:00, 2013-01-01 09:00:00, 2013-01-01 10:00:00, 2013-01-01 11:00:00, 2013-01-01 12:00:00, 2013-01-01 13:00:00, 2013-01-01 14:00:00, 2013-01-01 15:00:00, 2013-01-01 16:00:00, 2013-01-01 17:00:00, 2013-01-01 18:00:00, 2013-01-01 19:00:00, 2013-01-01 20:00:00, 2013-01-01 21:00:00, 2013-01-01 22:00:00, 2013-01-01 23:00:00, 2013-01-02 00:00:00, 2013-01-02 01:00:00, 2013-01-02 02:00:00, 2013-01-02 03:00:00, 2013-01-02 04:00:00, 2013-01-02 05:00:00, 2013-01-02 06:00:00, 2013-01-02 07:00:00, 2013-01-02 08:00:00, 2013-01-02 09:00:00, 2013-01-02 10:00:00, 2013-01-02 11:00:00, 2013-01-02 12:00:00, 2013-01-02 13:00:00, 2013-01-02 14:00:00, 2013-01-02 15:00:00, 2013-01-02 16:00:00, 2013-01-02 17:00:00, 2013-01-02 18:00:00, 2013-01-02 19:00:00, 2013-01-02 20:00:00, 2013-01-02 21:00:00

In [ ]:
#print(network.components)
#print(network.transformer_types)
#print(network.storage_units_t)
print("=== NON-EMPTY STORAGE_UNITS_T ATTRIBUTES ===")
for attr_name in dir(network.storage_units_t):
    if not attr_name.startswith('_'):  # Skip private attributes
        attr = getattr(network.storage_units_t, attr_name)
        if hasattr(attr, 'columns') and hasattr(attr, 'empty'):
            if not attr.empty:
                print(f"{attr_name}: {len(attr.columns)} storage units, {len(attr.index)} timesteps")
                print(f"  Storage units: {attr.columns.tolist()}")

=== NON-EMPTY GENERATORS_T ATTRIBUTES ===
inflow: 1 storage units, 8760 timesteps
  Storage units: ['ZA0 6 hydro']


In [ ]:
#after optimize
network.optimize()
print("=== NON-EMPTY STORAGE_UNITS_T ATTRIBUTES ===")
for attr_name in dir(network.storage_units_t):
    if not attr_name.startswith('_'):  # Skip private attributes
        attr = getattr(network.storage_units_t, attr_name)
        if hasattr(attr, 'columns') and hasattr(attr, 'empty'):
            if not attr.empty:
                print(f"{attr_name}: {len(attr.columns)} storage units, {len(attr.index)} timesteps")
                print(f"  Storage units: {attr.columns.tolist()}")

Index(['0', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6',
       '7', '8', '9', 'lines new ZA0 4 <-> ZA2 0 AC',
       'lines new ZA0 0 <-> ZA1 0 AC', 'lines new ZA0 0 <-> ZA3 0 AC'],
      dtype='object', name='Line')
Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA0 5', 'ZA0 6', 'ZA0 7',
       'ZA0 8', 'ZA0 9', 'ZA1 0', 'ZA2 0', 'ZA3 0'],
      dtype='object', name='Bus')
Writing continuous variables.: 100%|██████████| 7/7 [00:01<00:00,  5.50it/s]


=== NON-EMPTY STORAGE_UNITS_T ATTRIBUTES ===
inflow: 1 storage units, 8760 timesteps
  Storage units: ['ZA0 6 hydro']
p: 3 storage units, 8760 timesteps
  Storage units: ['ZA0 0 PHS', 'ZA0 5 PHS', 'ZA0 6 hydro']
p_dispatch: 3 storage units, 8760 timesteps
  Storage units: ['ZA0 0 PHS', 'ZA0 5 PHS', 'ZA0 6 hydro']
p_store: 3 storage units, 8760 timesteps
  Storage units: ['ZA0 0 PHS', 'ZA0 5 PHS', 'ZA0 6 hydro']
spill: 3 storage units, 8760 timesteps
  Storage units: ['ZA0 0 PHS', 'ZA0 5 PHS', 'ZA0 6 hydro']
state_of_charge: 3 storage units, 8760 timesteps
  Storage units: ['ZA0 0 PHS', 'ZA0 5 PHS', 'ZA0 6 hydro']


In [13]:
gdrive_base = '/content/drive/MyDrive/Colab_Notebooks/networks_1_year_connected'
network_file_path = os.path.join(gdrive_base, "elec_s_10_ec_lc1.0_1h.nc")
replacement_reward_offset=calculate_offset_k_initialization(network_file=network_file_path, input_dir="var_constraint_map")

Sampling 1000 random states to calculate offset k...


Index(['0', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6',
       '7', '8', '9', 'lines new ZA0 4 <-> ZA2 0 AC',
       'lines new ZA0 0 <-> ZA1 0 AC', 'lines new ZA0 0 <-> ZA3 0 AC'],
      dtype='object', name='Line')
Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA0 5', 'ZA0 6', 'ZA0 7',
       'ZA0 8', 'ZA0 9', 'ZA1 0', 'ZA2 0', 'ZA3 0'],
      dtype='object', name='Bus')
/usr/local/lib/python3.12/dist-packages/linopy/expressions.py:1861: FutureWarning:

In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): '_term' ('_term',) The recommendation is to set join explicitly for this case.



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded variable mapping from: /content/drive/MyDrive/Colab_Notebooks/var_constraint_map/elec_s_10_ec_lc1.0_1h_var_id_to_name.pkl
Loaded constraints from: /content/drive/MyDrive/Colab_Notebooks/var_constraint_map/elec_s_10_ec_lc1.0_1h_constraints.pkl
  Sample 0 failed: 'ZA0 0 PHS'
  Sample 1 failed: 'ZA0 0 PHS'
  Sample 2 failed: 'ZA0 0 PHS'
  Sample 3 failed: 'ZA0 0 PHS'
  Sample 4 failed: 'ZA0 0 PHS'


In [ ]:
import pypsa
import os

gdrive_base = '/content/drive/MyDrive/Colab_Notebooks/networks_1_year_connected'
network_file_path = os.path.join(gdrive_base, "elec_s_10_ec_lc1.0_1h.nc")

# Try to load the network and inspect it
try:
    network = pypsa.Network(network_file_path)
    print("Network loaded successfully")
    print(f"Network has {len(network.buses)} buses, {len(network.lines)} lines")
    print("Available components:", network.component_names)

    # Check if carriers are defined
    if hasattr(network, 'carriers'):
        print("Carriers:", network.carriers.index.tolist())
    else:
        print("No carriers defined")

except Exception as e:
    print(f"Error loading network: {e}")

In [ ]:
# Get the first snapshot
first_snapshot = network.snapshots[0]

# Dictionary to store storage unit time-varying attributes before lpf()
storage_units_t_before_lpf = {}

print("Capturing storage unit time-varying attributes before lpf()...")
for attr_name in dir(network.storage_units_t):
    if not attr_name.startswith('_'):  # Skip private attributes
        attr = getattr(network.storage_units_t, attr_name)
        if hasattr(attr, 'columns') and hasattr(attr, 'empty'):
            if not attr.empty:
                # Store the data for the first snapshot
                storage_units_t_before_lpf[attr_name] = attr.loc[first_snapshot].copy()
                print(f"  Captured '{attr_name}' for {first_snapshot}")

# Run linear power flow
print("\nRunning network.lpf()...")
network.lpf()
print("network.lpf() finished.")

# Dictionary to store storage unit time-varying attributes after lpf()
storage_units_t_after_lpf = {}

print("\nCapturing storage unit time-varying attributes after lpf()...")
for attr_name in dir(network.storage_units_t):
    if not attr_name.startswith('_'):  # Skip private attributes
        attr = getattr(network.storage_units_t, attr_name)
        if hasattr(attr, 'columns') and hasattr(attr, 'empty'):
            if not attr.empty:
                # Store the data for the first snapshot
                storage_units_t_after_lpf[attr_name] = attr.loc[first_snapshot].copy()
                print(f"  Captured '{attr_name}' for {first_snapshot}")

print("\nComparing storage unit time-varying attributes for the first snapshot...")
attributes_changed = False
for attr_name, before_data in storage_units_t_before_lpf.items():
    if attr_name in storage_units_t_after_lpf:
        after_data = storage_units_t_after_lpf[attr_name]
        # Compare the dataframes/series for equality
        if not before_data.equals(after_data):
            print(f"  '{attr_name}' CHANGED after lpf() for snapshot {first_snapshot}")
            attributes_changed = True
        else:
            print(f"  '{attr_name}' remained the SAME after lpf() for snapshot {first_snapshot}")
    else:
        print(f"  '{attr_name}' not found after lpf()")

if not attributes_changed:
    print("\nAll captured storage unit time-varying attributes for the first snapshot remained the same after lpf().")
else:
    print("\nSome storage unit time-varying attributes for the first snapshot changed after lpf().")

Capturing storage unit time-varying attributes before lpf()...
  Captured 'inflow' for 2013-01-01 00:00:00
  Captured 'p' for 2013-01-01 00:00:00
  Captured 'p_dispatch' for 2013-01-01 00:00:00
  Captured 'p_store' for 2013-01-01 00:00:00
  Captured 'spill' for 2013-01-01 00:00:00
  Captured 'state_of_charge' for 2013-01-01 00:00:00

Running network.lpf()...
network.lpf() finished.

Capturing storage unit time-varying attributes after lpf()...
  Captured 'inflow' for 2013-01-01 00:00:00
  Captured 'p' for 2013-01-01 00:00:00
  Captured 'p_dispatch' for 2013-01-01 00:00:00
  Captured 'p_store' for 2013-01-01 00:00:00
  Captured 'spill' for 2013-01-01 00:00:00
  Captured 'state_of_charge' for 2013-01-01 00:00:00

Comparing storage unit time-varying attributes for the first snapshot...
  'inflow' remained the SAME after lpf() for snapshot 2013-01-01 00:00:00
  'p' remained the SAME after lpf() for snapshot 2013-01-01 00:00:00
  'p_dispatch' remained the SAME after lpf() for snapshot 2013-

In [ ]:
network=pypsa.Network("networks_1_week_connected/elec_s_10_ec_lc1.0_1h.nc")
network.optimize()
generators_with_p_max_pu_t = network.generators_t.p_max_pu.columns
# Filter network.generators to find those NOT in the list above. These are the dispatchable generators
generators_without_p_max_pu_t = network.generators[~network.generators.index.isin(generators_with_p_max_pu_t)].index
non_slack_gens = network.generators[(network.generators.control != "Slack") & (~network.generators.index.isin(generators_with_p_max_pu_t))].index #must explicitly search again, can't reference results of last query since this may have changed since choosing slack generator
non_slack_names = list(non_slack_gens)

# Set p_set to all zeros for the non_slack_gens

network.generators_t.p_set.loc[:, non_slack_names] = pd.DataFrame(
    0.0,
    index=network.snapshots,
    columns=non_slack_names
    )


for i, gen_name in enumerate(non_slack_names):
  print(network.generators.p_nom.loc[gen_name])
  #print(network.generators_t.p_set.iloc[0].loc[gen_name])

Index(['0', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6',
       '7', '8', '9', 'lines new ZA0 4 <-> ZA2 0 AC',
       'lines new ZA0 0 <-> ZA1 0 AC', 'lines new ZA0 0 <-> ZA3 0 AC'],
      dtype='object', name='Line')
Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA0 5', 'ZA0 6', 'ZA0 7',
       'ZA0 8', 'ZA0 9', 'ZA1 0', 'ZA2 0', 'ZA3 0'],
      dtype='object', name='Bus')
Writing continuous variables.: 100%|██████████| 7/7 [00:01<00:00,  5.40it/s]


740.0
10.1
2352.0
301.7403409455683
38589.96217393791
1338.3
1940.0
670.0
72.3
335.0
3.5
242.46941957611483


In [ ]:
# Get the first snapshot
first_snapshot = network.snapshots[0]
snapshot_idx=0

# Dictionary to store storage unit time-varying attributes before lpf()
storage_units_t_before_lpf = {}

print("Capturing storage unit time-varying attributes before lpf()...")
for attr_name in dir(network.storage_units_t):
    if not attr_name.startswith('_'):  # Skip private attributes
        attr = getattr(network.storage_units_t, attr_name)
        if hasattr(attr, 'columns') and hasattr(attr, 'empty'):
            if not attr.empty:
                # Store the data for the first snapshot
                storage_units_t_before_lpf[attr_name] = attr.loc[first_snapshot].copy()
                print(f"  Captured '{attr_name}' for {first_snapshot}")

# Run linear power flow
print("\nRunning network.lpf()...")
for i, gen_name in enumerate(non_slack_names):
            network.generators_t.p_set.iloc[0].loc[gen_name] = network.generators.p_nom.loc[gen_name]
network.lpf()
print("network.lpf() finished.")

# Dictionary to store storage unit time-varying attributes after lpf()
storage_units_t_after_lpf = {}

print("\nCapturing storage unit time-varying attributes after lpf()...")
for attr_name in dir(network.storage_units_t):
    if not attr_name.startswith('_'):  # Skip private attributes
        attr = getattr(network.storage_units_t, attr_name)
        if hasattr(attr, 'columns') and hasattr(attr, 'empty'):
            if not attr.empty:
                # Store the data for the first snapshot
                storage_units_t_after_lpf[attr_name] = attr.loc[first_snapshot].copy()
                print(f"  Captured '{attr_name}' for {first_snapshot}")

print("\nComparing storage unit time-varying attributes for the first snapshot...")
attributes_changed = False
for attr_name, before_data in storage_units_t_before_lpf.items():
    if attr_name in storage_units_t_after_lpf:
        after_data = storage_units_t_after_lpf[attr_name]
        # Compare the dataframes/series for equality
        if not before_data.equals(after_data):
            print(f"  '{attr_name}' CHANGED after lpf() for snapshot {first_snapshot}")
            attributes_changed = True
        else:
            print(f"  '{attr_name}' remained the SAME after lpf() for snapshot {first_snapshot}")
    else:
        print(f"  '{attr_name}' not found after lpf()")

if not attributes_changed:
    print("\nAll captured storage unit time-varying attributes for the first snapshot remained the same after lpf().")
else:
    print("\nSome storage unit time-varying attributes for the first snapshot changed after lpf().")

Capturing storage unit time-varying attributes before lpf()...
  Captured 'inflow' for 2013-01-01 00:00:00
  Captured 'p' for 2013-01-01 00:00:00
  Captured 'p_dispatch' for 2013-01-01 00:00:00
  Captured 'p_store' for 2013-01-01 00:00:00
  Captured 'spill' for 2013-01-01 00:00:00
  Captured 'state_of_charge' for 2013-01-01 00:00:00

Running network.lpf()...


/tmp/ipython-input-1794625506.py:21: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/tmp/ipython-input-1794625506.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docum

network.lpf() finished.

Capturing storage unit time-varying attributes after lpf()...
  Captured 'inflow' for 2013-01-01 00:00:00
  Captured 'p' for 2013-01-01 00:00:00
  Captured 'p_dispatch' for 2013-01-01 00:00:00
  Captured 'p_store' for 2013-01-01 00:00:00
  Captured 'spill' for 2013-01-01 00:00:00
  Captured 'state_of_charge' for 2013-01-01 00:00:00

Comparing storage unit time-varying attributes for the first snapshot...
  'inflow' remained the SAME after lpf() for snapshot 2013-01-01 00:00:00
  'p' remained the SAME after lpf() for snapshot 2013-01-01 00:00:00
  'p_dispatch' remained the SAME after lpf() for snapshot 2013-01-01 00:00:00
  'p_store' remained the SAME after lpf() for snapshot 2013-01-01 00:00:00
  'spill' remained the SAME after lpf() for snapshot 2013-01-01 00:00:00
  'state_of_charge' remained the SAME after lpf() for snapshot 2013-01-01 00:00:00

All captured storage unit time-varying attributes for the first snapshot remained the same after lpf().


In [ ]:
generators_with_p_max_pu_t = network.generators_t.p_max_pu.columns
print(generators_with_p_max_pu_t)
#print(network.generators_t.loc[:,generators_with_p_max_pu_t])

Index(['ZA0 0 onwind', 'ZA0 0 ror', 'ZA0 0 solar', 'ZA0 1 offwind-ac',
       'ZA0 1 offwind-dc', 'ZA0 1 onwind', 'ZA0 1 solar', 'ZA0 2 onwind',
       'ZA0 2 ror', 'ZA0 2 solar', 'ZA0 3 onwind', 'ZA0 3 ror', 'ZA0 3 solar',
       'ZA0 4 onwind', 'ZA0 4 solar', 'ZA0 5 offwind-ac', 'ZA0 5 offwind-dc',
       'ZA0 5 onwind', 'ZA0 5 solar', 'ZA0 6 onwind', 'ZA0 6 solar',
       'ZA0 7 offwind-ac', 'ZA0 7 offwind-dc', 'ZA0 7 onwind', 'ZA0 7 solar',
       'ZA0 8 offwind-ac', 'ZA0 8 offwind-dc', 'ZA0 8 onwind', 'ZA0 8 ror',
       'ZA0 8 solar', 'ZA0 9 onwind', 'ZA0 9 solar', 'ZA1 0 offwind-ac',
       'ZA1 0 offwind-dc', 'ZA1 0 onwind', 'ZA1 0 solar', 'ZA2 0 onwind',
       'ZA2 0 solar', 'ZA3 0 offwind-ac', 'ZA3 0 offwind-dc', 'ZA3 0 onwind',
       'ZA3 0 solar'],
      dtype='object', name='Generator')


In [ ]:
network=pypsa.Network("networks_1_week_connected/elec_s_10_ec_lc1.0_1h.nc")
network.optimize()

Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA0 5', 'ZA0 6', 'ZA0 7',
       'ZA0 8', 'ZA0 9', 'ZA1 0', 'ZA2 0', 'ZA3 0'],
      dtype='object', name='Bus')
Index(['0', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6',
       '7', '8', '9', 'lines new ZA0 4 <-> ZA2 0 AC',
       'lines new ZA0 0 <-> ZA1 0 AC', 'lines new ZA0 0 <-> ZA3 0 AC'],
      dtype='object', name='Line')
Writing continuous variables.: 100%|██████████| 7/7 [00:01<00:00,  5.42it/s]


('ok', 'optimal')

In [ ]:
# Get all DataFrame attributes from generators_t
generator_attrs = {
    attr_name: getattr(network.generators_t, attr_name)
    for attr_name in dir(network.generators_t)
    if not attr_name.startswith('_') and
       hasattr(getattr(network.generators_t, attr_name), 'columns')
}

# Filter each attribute for your target generators
filtered_data = {}
for attr_name, attr_df in generator_attrs.items():
    available_generators = attr_df.columns.intersection(generators_with_p_max_pu_t)
    if not available_generators.empty:
        filtered_data[attr_name] = attr_df.loc[:, available_generators]
        print(f"{attr_name}: {len(available_generators)} generators")

p: 42 generators
p_max_pu: 42 generators


In [ ]:
# Get all DataFrame attributes from generators_t
generator_attrs = {
    attr_name: getattr(network.generators_t, attr_name)
    for attr_name in dir(network.generators_t)
    if not attr_name.startswith('_') and
       hasattr(getattr(network.generators_t, attr_name), 'columns')
}

# Filter each attribute for your target generators - FIRST SNAPSHOT ONLY
filtered_data_first_snapshot = {}
for attr_name, attr_df in generator_attrs.items():
    available_generators = attr_df.columns.intersection(generators_with_p_max_pu_t)
    if not available_generators.empty:
        # Get only the first snapshot (first row)
        filtered_data_first_snapshot[attr_name] = attr_df.iloc[0].loc[available_generators]
        print(f"{attr_name}: {len(available_generators)} generators at first snapshot")
        print(filtered_data_first_snapshot[attr_name])
        print("-" * 30)

p: 42 generators at first snapshot
Generator
ZA0 0 onwind          2.006375
ZA0 0 ror             2.228640
ZA0 0 solar          -0.000000
ZA0 1 offwind-ac     -0.000000
ZA0 1 offwind-dc     -0.000000
ZA0 1 onwind        120.497676
ZA0 1 solar          -0.000000
ZA0 2 onwind        370.640179
ZA0 2 ror            12.570000
ZA0 2 solar          -0.000000
ZA0 3 onwind          0.784162
ZA0 3 ror             2.231386
ZA0 3 solar          -0.000000
ZA0 4 onwind          3.252372
ZA0 4 solar          -0.000000
ZA0 5 offwind-ac     -0.000000
ZA0 5 offwind-dc     -0.000000
ZA0 5 onwind        261.085891
ZA0 5 solar          -0.000000
ZA0 6 onwind        155.181726
ZA0 6 solar          -0.000000
ZA0 7 offwind-ac     -0.000000
ZA0 7 offwind-dc     -0.000000
ZA0 7 onwind         18.785402
ZA0 7 solar          -0.000000
ZA0 8 offwind-ac     -0.000000
ZA0 8 offwind-dc     -0.000000
ZA0 8 onwind         40.215754
ZA0 8 ror             6.591051
ZA0 8 solar          -0.000000
ZA0 9 onwind         66.1

In [ ]:
#check the same as above cell for non-dispatchable generators:
# Get the first snapshot
first_snapshot = network.snapshots[0]
snapshot_idx=0

generators_with_p_max_pu_t = network.generators_t.p_max_pu.columns
# Filter network.generators to find those NOT in the list above. These are the dispatchable generators
generators_without_p_max_pu_t = network.generators[~network.generators.index.isin(generators_with_p_max_pu_t)].index
non_slack_gens = network.generators[(network.generators.control != "Slack") & (~network.generators.index.isin(generators_with_p_max_pu_t))].index #must explicitly search again, can't reference results of last query since this may have changed since choosing slack generator
non_slack_names = list(non_slack_gens)

# Set p_set to all zeros for the non_slack_gens

network.generators_t.p_set.loc[:, non_slack_names] = pd.DataFrame(
    0.0,
    index=network.snapshots,
    columns=non_slack_names
    )

print("Capturing storage unit time-varying attributes before lpf()...")
# Get all DataFrame attributes from generators_t
generator_attrs = {
    attr_name: getattr(network.generators_t, attr_name)
    for attr_name in dir(network.generators_t)
    if not attr_name.startswith('_') and
       hasattr(getattr(network.generators_t, attr_name), 'columns')
}

# Filter each attribute for your target generators - FIRST SNAPSHOT ONLY
filtered_data_first_snapshot = {}
for attr_name, attr_df in generator_attrs.items():
    available_generators = attr_df.columns.intersection(generators_with_p_max_pu_t)
    if not available_generators.empty:
        # Get only the first snapshot (first row)
        filtered_data_first_snapshot[attr_name] = attr_df.iloc[0].loc[available_generators]
        print(f"{attr_name}: {len(available_generators)} generators at first snapshot")
        print(filtered_data_first_snapshot[attr_name])
        print("-" * 30)

# Run linear power flow
print("\nRunning network.lpf()...")
for i, gen_name in enumerate(non_slack_names):
            network.generators_t.p_set.iloc[0].loc[gen_name] = network.generators.p_nom.loc[gen_name]
network.lpf()
print("network.lpf() finished.")

# Get all DataFrame attributes from generators_t
generator_attrs = {
    attr_name: getattr(network.generators_t, attr_name)
    for attr_name in dir(network.generators_t)
    if not attr_name.startswith('_') and
       hasattr(getattr(network.generators_t, attr_name), 'columns')
}

# Filter each attribute for your target generators - FIRST SNAPSHOT ONLY
filtered_data_first_snapshot_after = {}
for attr_name, attr_df in generator_attrs.items():
    available_generators = attr_df.columns.intersection(generators_with_p_max_pu_t)
    if not available_generators.empty:
        # Get only the first snapshot (first row)
        filtered_data_first_snapshot_after[attr_name] = attr_df.iloc[0].loc[available_generators]
        print(f"{attr_name}: {len(available_generators)} generators at first snapshot")
        print(filtered_data_first_snapshot_after[attr_name])
        print("-" * 30)

print("\nComparing storage unit time-varying attributes for the first snapshot...")
attributes_changed = False
for attr_name, before_data in filtered_data_first_snapshot.items():
    if attr_name in filtered_data_first_snapshot_after:
        after_data = filtered_data_first_snapshot_after[attr_name]
        # Compare the dataframes/series for equality
        if not before_data.equals(after_data):
            print(f"  '{attr_name}' CHANGED after lpf() for snapshot {first_snapshot}")
            attributes_changed = True
        else:
            print(f"  '{attr_name}' remained the SAME after lpf() for snapshot {first_snapshot}")
    else:
        print(f"  '{attr_name}' not found after lpf()")

if not attributes_changed:
    print("\nAll captured storage unit time-varying attributes for the first snapshot remained the same after lpf().")
else:
    print("\nSome storage unit time-varying attributes for the first snapshot changed after lpf().")

Capturing storage unit time-varying attributes before lpf()...
p: 42 generators at first snapshot
Generator
ZA0 0 onwind          2.006375
ZA0 0 ror             2.228640
ZA0 0 solar          -0.000000
ZA0 1 offwind-ac     -0.000000
ZA0 1 offwind-dc     -0.000000
ZA0 1 onwind        120.497676
ZA0 1 solar          -0.000000
ZA0 2 onwind        370.640179
ZA0 2 ror            12.570000
ZA0 2 solar          -0.000000
ZA0 3 onwind          0.784162
ZA0 3 ror             2.231386
ZA0 3 solar          -0.000000
ZA0 4 onwind          3.252372
ZA0 4 solar          -0.000000
ZA0 5 offwind-ac     -0.000000
ZA0 5 offwind-dc     -0.000000
ZA0 5 onwind        261.085891
ZA0 5 solar          -0.000000
ZA0 6 onwind        155.181726
ZA0 6 solar          -0.000000
ZA0 7 offwind-ac     -0.000000
ZA0 7 offwind-dc     -0.000000
ZA0 7 onwind         18.785402
ZA0 7 solar          -0.000000
ZA0 8 offwind-ac     -0.000000
ZA0 8 offwind-dc     -0.000000
ZA0 8 onwind         40.215754
ZA0 8 ror             6.

/tmp/ipython-input-105154726.py:43: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


/tmp/ipython-input-105154726.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documen

network.lpf() finished.
p: 42 generators at first snapshot
Generator
ZA0 0 onwind        0.0
ZA0 0 ror           0.0
ZA0 0 solar         0.0
ZA0 1 offwind-ac    0.0
ZA0 1 offwind-dc    0.0
ZA0 1 onwind        0.0
ZA0 1 solar         0.0
ZA0 2 onwind        0.0
ZA0 2 ror           0.0
ZA0 2 solar         0.0
ZA0 3 onwind        0.0
ZA0 3 ror           0.0
ZA0 3 solar         0.0
ZA0 4 onwind        0.0
ZA0 4 solar         0.0
ZA0 5 offwind-ac    0.0
ZA0 5 offwind-dc    0.0
ZA0 5 onwind        0.0
ZA0 5 solar         0.0
ZA0 6 onwind        0.0
ZA0 6 solar         0.0
ZA0 7 offwind-ac    0.0
ZA0 7 offwind-dc    0.0
ZA0 7 onwind        0.0
ZA0 7 solar         0.0
ZA0 8 offwind-ac    0.0
ZA0 8 offwind-dc    0.0
ZA0 8 onwind        0.0
ZA0 8 ror           0.0
ZA0 8 solar         0.0
ZA0 9 onwind        0.0
ZA0 9 solar         0.0
ZA1 0 offwind-ac    0.0
ZA1 0 offwind-dc    0.0
ZA1 0 onwind        0.0
ZA1 0 solar         0.0
ZA2 0 onwind        0.0
ZA2 0 solar         0.0
ZA3 0 offwind-ac   